In [1]:
##IMPORTS
import sys
sys.path.append("/workspace/QuantLab")

import pandas as pd
from RQCharts.main import RQCharts
from RQAlpha import RTools
from talib import abstract
import matplotlib.pyplot as plt
import numpy as np
import random
from backtesting import Backtest

/workspace/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
### Initial construction of random tickers list of df with indicators. 

print(f"In this cell choose the indicators needed and percentage of total data to test on")

indicators = ["ATR", "RSI", "BBANDS", "STD", "SMA", "EMA"]
resolution = "5m"
sma_ema_days = [14, 50, 100, 150, 200]
percent = 0.40

RQCharts = RQCharts()
RTools = RTools()

stored_symbols = []
symbol_df_dict = {}

#IF resolution < Daily it will retrieve symbols already colelcted in minute data else in daily data
if resolution in ["minute", "5m", "10m", "15m", "30m", "hourly"]:
    db_response= RQCharts.conn.insert_new_command("SELECT DISTINCT symbol from stocks.minute;")
    for element in db_response:
        stored_symbols.append(element[0])
else:
    stored_symbols = RQCharts.watchlists.get_watched()


random_symbols = random.sample(stored_symbols, int(percent * len(stored_symbols))) 

if not any(ticker in random_symbols for ticker in ["SPY", "QQQ", "TSLA"]):
    missing_ticker = next(ticker for ticker in ["SPY", "QQQ", "TSLA"] if ticker not in random_symbols)
    random_symbols.append(missing_ticker) 

for symbol in random_symbols:
    df = RQCharts.add_equity(symbol, resolution=resolution)
    df.dropna(subset=['Close'], inplace=True)  # Removes rows where 'close' is NaN
    df=df[df['High']!=df['Low']]
    df['ATR'] = RTools.ATR(df, window = 20)
    df['RSI']= RTools.RSI(df, period =15)
    df['VWAP']= RTools.VWAP(df, window =15)
    RTools.BBANDS(df= df, window=20, std = 2.0)
    
    for days in sma_ema_days:
        df[f"EMA_{days}"] = RTools.EMA(df, window = days)
        df[f"SMA_{days}"] = RTools.SMA(df, window = days)
    df.dropna()

    symbol_df_dict[symbol]= df



In [ ]:
#Signal STRATEGY: define your signal generating function here 

In [ ]:
#SIGNAL PLOTTING for df wiht [ TotalSignal, BBANDS, VWAP]
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

st=16500
dfpl = df[st:st+5200]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                           line=dict(color='blue', width=1), 
                           name="VWAP"), 
                go.Scatter(x=dfpl.index, y=dfpl['lowerband'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['upperband'], 
                           line=dict(color='green', width=1), 
                           name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")
fig.show()

In [14]:
#Backtesting Layout: define your backtesting class here
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL():
    return df.TotalSignal

class MyStrat(Strategy):
    TPSLRatio = 2
    initsize = 0.5 

    def init(self):
        super().init()
        self.mysize = self.initsize  
        self.signal1 = self.I(SIGNAL)

    def next(self):
       

In [ ]:
## Run Optimize and sort your Backtest results
strat_name = 
str_strat_name = str(strat_name)
cash = 10000
margin = 1/10
commission= .01
results_df = {}  

for symbol, df in symbol_df_dict.items():  
    print(f"Running backtest for {symbol}...")  

    # Run the backtest with optimization
    bt = Backtest(
        df, 
        strat_name, 
        cash=cash, 
        margin=margin, 
        commission=commission
    )
    
    # Optimize parameters
    stat, heatmap = bt.optimize(
        TPSLRatio=np.arange(1.5, 4, 0.25).tolist(),
        initsize=np.arange(0.4, 1.00, 0.1).tolist(),
        maximize="Sharpe Ratio",
        method="grid",
        return_heatmap=True,
        return_optimization=False,
        random_state=None,
    )

    # Plot backtest results
    bt.plot(
        filename=f"Backtest_Results/{symbol}_{str_strat_name}.html",
        plot_equity=True,
        plot_pl=True,
        plot_volume=True,
        plot_trades=True,
        resample=True,
        open_browser=False,
    )

    # Store results
    results_df[symbol] = {"stats": stat, "heatmap": heatmap}




In [ ]:
#ANALYZE RETURNS
# Initialize an empty list to store summary metrics for each symbol
summary_stats = []

# Analyze each symbol's stats and heatmap
for symbol, result in results_df.items():
    stats = result["stats"]
    heatmap = result["heatmap"]

    # Extract important metrics from stats
    metrics = {
        "Symbol": symbol,
        "Sharpe Ratio": stats["Sharpe Ratio"],
        "CAGR [%]": stats["CAGR [%]"],
        "Max. Drawdown [%]": stats["Max. Drawdown [%]"],
        "Win Rate [%]": stats["Win Rate [%]"],
        "Best Trade [%]": stats["Best Trade [%]"],
        "Worst Trade [%]": stats["Worst Trade [%]"],
        "Trades": stats["# Trades"],
    }

    # Append metrics to the list
    summary_stats.append(metrics)

    # Plot the heatmap for this symbol
    plt.figure(figsize=(10, 8))
    sns.heatmap(heatmap.unstack(level=1), annot=False, cmap="coolwarm", center=0)
    plt.title(f"Heatmap for {symbol} (Sharpe Ratio)")
    plt.xlabel("initsize")
    plt.ylabel("TPSLRatio")
    plt.show()

# Convert the summary_stats list to a DataFrame
summary_stats_df = pd.DataFrame(summary_stats)

# Analyze the summary_stats DataFrame
print("\nSummary of all symbols:")
print(summary_stats_df)

# Sort by Sharpe Ratio for the best strategies
best_strategies = summary_stats_df.sort_values(by="Sharpe Ratio", ascending=False)
print("\nTop Strategies by Sharpe Ratio:")
print(best_strategies.head(10))

# Save summary stats to CSV
summary_stats_df.to_csv("strategy_summary.csv", index=False)
print("\nSummary stats saved to 'strategy_summary.csv'.")

# Plot comparisons for key metrics
plt.figure(figsize=(12, 6))
sns.barplot(x="Symbol", y="Sharpe Ratio", data=summary_stats_df)
plt.title("Sharpe Ratio Comparison Across Symbols")
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(x="Symbol", y="CAGR [%]", data=summary_stats_df)
plt.title("CAGR Comparison Across Symbols")
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(x="Symbol", y="Max. Drawdown [%]", data=summary_stats_df)
plt.title("Max Drawdown Comparison Across Symbols")
plt.show()

